# Defining cohorts with sgkit

In [63]:
%run setup.ipynb

In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
from dask.diagnostics import ProgressBar
import numpy as np
import sgkit as sg
import xarray as xr

In [66]:
ds = xr.open_zarr(str(here() / 'data/sgkit/ag1000g_rechunked4.zarr'), concat_characters=False)
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

Get all the cohort (population) IDs

In [67]:
cohort_ids = list(pop_defs.keys())
cohort_ids

['ao_col',
 'bf_col',
 'bf_gam',
 'ci_col',
 'cm_sav_gam',
 'fr_gam',
 'ga_gam',
 'gh_col',
 'gh_gam',
 'gm',
 'gn_gam',
 'gq_gam',
 'gw',
 'ke',
 'ug_gam']

In [68]:
ds["cohort_id"] = xr.DataArray(cohort_ids, dims="cohorts")
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    cohort_id           (cohorts) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

Sample metadata is in the `df_samples` dataframe, so we can use that to produce a mapping from sample to cohort

In [69]:
sample_cohorts = np.full_like(ds.sample_id.values, -1, dtype=np.int8)
for i, pop in enumerate(cohort_ids):
    pop_query = (
            pop_defs[pop]['query']
            .replace('region', 'location')
            .replace('Gado-Badzere', 'Gado Badzere')
            .replace('Zembe-Borongo', 'Zembe Borongo')
    )
    loc_pop = df_samples.query(pop_query).index.values
    sample_cohorts[loc_pop] = i
sample_cohorts

array([7, 7, 7, ..., 3, 3, 3], dtype=int8)

Add `sample_cohort` to the dataset

In [70]:
ds["sample_cohort"] = xr.DataArray(sample_cohorts, dims="samples")
ds

<xarray.Dataset>
Dimensions:             (alleles: 4, cohorts: 15, ploidy: 2, samples: 1142, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(65536, 1142, 2), meta=np.ndarray>
    sample_id           (samples) <U1 dask.array<chunksize=(1142,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(65536, 4), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(65536,), meta=np.ndarray>
    cohort_id           (cohorts) <U10 'ao_col' 'bf_col' ... 'ke' 'ug_gam'
    sample_cohort       (samples) int8 7 7 7 7 7 8 7 8 7 7 ... 3 3 3 3 3 3 3 3 3
Attributes:
    contigs:                    ['2L', '2R', '3L', '3R', 'X']
    max_variant_allele_length:  1

Some samples are not in any of the named cohorts, and have -1 in the `sample_cohort` variable. These are ignored in cohort allele counts.

In [71]:
#ds = ds.isel(samples=ds.sample_cohort >= 0)
#ds

Count cohort alleles

In [72]:
cac = sg.count_cohort_alleles(ds, merge=False)
cac

<xarray.Dataset>
Dimensions:              (alleles: 4, cohorts: 15, variants: 57837885)
Dimensions without coordinates: alleles, cohorts, variants
Data variables:
    cohort_allele_count  (variants, cohorts, alleles) int32 dask.array<chunksize=(65536, 15, 4), meta=np.ndarray>

Save

In [73]:
with ProgressBar():
    cac.to_zarr(str(here() / 'data/sgkit/ag1000g_cohort_allele_count.zarr'))

[########################################] | 100% Completed |  7min 49.8s


Notes

In [48]:
np.where(sample_cohorts == -1)

(array([ 550,  551,  552,  553,  554,  555,  556,  557,  558,  559,  560,
         561,  562,  563,  660,  661,  662,  663,  664,  665,  666,  667,
         668,  669,  670,  671,  672,  673,  674,  675,  676,  677,  678,
         679,  680,  681,  682,  683,  684,  685,  686,  687,  688,  689,
         690,  691,  692,  693,  694,  695,  696,  697,  698,  699,  700,
         701,  702,  703,  704,  705,  706,  707,  708,  709,  710,  711,
         712,  713,  714,  715,  716,  717,  718,  719,  720,  721,  722,
         723,  724,  725,  726,  727,  728,  729,  730,  731,  732,  733,
         734,  735,  736,  737,  738,  739,  740,  741,  742,  743,  744,
         745,  746,  747,  748,  749,  750,  751,  752,  753,  754,  755,
         756,  757,  758,  759,  760,  761,  762,  763,  764,  765,  766,
         767,  768,  769,  770,  771,  772,  773,  774,  775,  776,  777,
         778,  779,  780,  781,  782,  783,  784,  785,  786,  787,  788,
         789,  790,  791,  792,  793, 

In [32]:
df_samples.iloc[[550]]

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
550,AN0007-C,CM0901778,CMgam,Cameroon,Mayos,Mayos,Nora Besansky,Reidenbach,2009,S,F,102380716,30.44,ERS224381,4.341,13.558
